<a href="https://colab.research.google.com/github/HARASEON/Deep_Learning_tensorflow/blob/main/Day2_DeepLearning_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Iris prediction with tensorflow
* Iris data 특징
* 붓꽃의 3가지 종류를 기록한 데이터
* label: 붓꽃의 종류인 traget -> 0,1,2
* 150개 데이터


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.6f}".format(x)})


# load dataset
train_ds = tfds.load('iris', split = 'train[:80%]')
test_ds = tfds.load('iris', split= 'train[-20%:]')

# data preprocessing
# label : 3ea
def preprocess(features):
    f,l = features['features'], tf.one_hot(features['label'], 3)
    return f,l

train_data = train_ds.map(preprocess).batch(10)
valid_data = test_ds.map(preprocess).batch(10)

# design model
model = Sequential([
    Dense(6, activation="relu"),
    Dense(5, activation="relu"),
    Dense(3, activation="softmax")
])
  
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc']) 

# model fitting
model.fit(train_data,
    validation_data=(valid_data),
    verbose=0,
    epochs=50)

model.save("iris.h5")
model.summary()

# model Use
pred_test = model.predict(valid_data)
print(pred_test[2])

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 6)                 30        
                                                                 
 dense_22 (Dense)            (None, 5)                 35        
                                                                 
 dense_23 (Dense)            (None, 3)                 18        
                                                                 
Total params: 83
Trainable params: 83
Non-trainable params: 0
_________________________________________________________________
3/3 [==============================] - 0s 3ms/step
[0.012088 0.504675 0.483237]


#Iris dataset이 CNN을 사용할 수 없는 이유:
image dataset이 아니라서 CNN은 이미지 데이터셋 분류를 위해 설계된 알고리즘이다.

# Chat GPT with iris dataset
# No CNN
* Goal:val_lose <= 0.13
* Prompt:please, code efficient, fast and best performing program with minimum val loss value using python tensorflow
from iris dataset from tensorflow_datasets
to perform classification
use only dense layers 
use dropout
use batch normalization
use checkpoint with save best only, and save weight only 
use early stopping with patient 10
Use reduce lr on plateau
set verbose = 1
do not use data augmentation
and other method to make val loss value lower than 0.13.

* Code detail:
This code loads the iris dataset using TensorFlow Datasets, defines a simple fully connected neural network with batch normalization and dropout, compiles the model with an Adam optimizer and sparse categorical cross-entropy loss function, and trains the model using early stopping, learning rate reduction on plateau and model checkpointing to save the best model. Finally, it evaluates the best model on the validation set and prints the validation loss and accuracy.

**결과: 에러가 떴다!
KeyError: 'validation'**

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Load the iris dataset from tensorflow_datasets
data, info = tfds.load("iris", with_info=True, as_supervised=True)
train_data = data['train']
val_data = data['validation']

# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Define the callbacks
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', 
                                                         save_best_only=True,
                                                         save_weights_only=True,
                                                         monitor='val_loss')
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                                          patience=5, min_lr=0.0001)

# Train the model
history = model.fit(train_data.batch(32), 
                    epochs=100, 
                    validation_data=val_data.batch(32),
                    callbacks=[checkpoint_callback, early_stop_callback, reduce_lr_callback],
                    verbose=1)

# Load the best saved model
model.load_weights('best_model.h5')

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_data.batch(32), verbose=0)

print("Validation loss:", val_loss)
print("Validation accuracy:", val_acc)


KeyError: ignored

**따라서, 다시 prompt에 debug하라는 명령을 내렸다**

chatGPT: Apologies for my mistake in the previous code. The data dictionary returned by tfds.load("iris", with_info=True, as_supervised=True) does not contain a validation key.

** ChatGPT는 이름만 바꿨다.**



In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the iris dataset from tensorflow_datasets
data, info = tfds.load("iris", with_info=True, as_supervised=True)
train_data = data['train']
val_data = data['test']

# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Define the callbacks
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', 
                                                         save_best_only=True,
                                                         save_weights_only=True,
                                                         monitor='val_loss')
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                                          patience=5, min_lr=0.0001)

# Train the model
history = model.fit(train_data.batch(32), 
                    epochs=100, 
                    validation_data=val_data.batch(32),
                    callbacks=[checkpoint_callback, early_stop_callback, reduce_lr_callback],
                    verbose=1)

# Load the best saved model
model.load_weights('best_model.h5')

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_data.batch(32), verbose=0)

print("Validation loss:", val_loss)
print("Validation accuracy:", val_acc)


KeyError: ignored